In [1]:
using JLD2
using FileIO
using PyPlot
using Statistics
using StatsBase 
using Distributions
using LinearAlgebra
using HDF5
using KernelDensity
using IntervalSets
using Random, ArraysOfArrays
using ValueShapes
using Measurements
using BAT
using DSP
using ForwardDiff

In [2]:
FPATH = "../../data/sampling_results/Benchmark-5/"

function marg_mean_shaped(name)
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = mean(smpl_tmp)
    return smpl_tmp
end

function marg_std_shaped(name)
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = std(smpl_tmp)
    return smpl_tmp
end


marg_std_shaped (generic function with 1 method)

In [3]:
β1 = 0.015
β2 = 0.0077 # (0.8/30/sqrt(12))
β3 = 0.0058 # (0.8/40/sqrt(12))

# flat prior for the pixel size
prior = NamedTupleDist(
        tr_size = [truncated(Normal(0.2, 0.04), 0.06, 0.19), truncated(Normal(0.2, 0.04), 0.06, 0.19)],
        tr_size_2 = [truncated(Normal(0.2, 0.04), 0.06, 0.19), truncated(Normal(0.2, 0.04), 0.06, 0.19)],
        ang_spr = [truncated(Normal(4.0, 2.0), 4.0, 7.0), truncated(Normal(4.0, 2.0), 4.0, 7.0)],
        ang_spr_2 = [truncated(Normal(4.0, 2.0), 1.0, 4.0), truncated(Normal(4.0, 2.0), 1.0, 4.0)],
        mixt_pow =  0.50 .. 1.0 ,
        waist = [truncated(Normal(2.9, 0.03), 2.65, 3.5)],
        waist_2 = [truncated(Normal(2.9, 0.03), 2.65, 3.5)], # 11
        algmx = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
        algmy = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
        cam4_ped = 4.0 .. 40.0,
        cam4_light_fluct = 1.0 .. 3.0,
        cam4_light_amp = 1.6 .. 9.9, 
        resx = [1, 1, 1], # 23, 24, 25, 
        resy = [1, 1, 1], # 26,27, 28, 
        cam4_resx = truncated(Normal(3, 1.5), 0, Inf),
        cam4_resy = truncated(Normal(3, 1.5), 0, Inf), 
        psx = [27.1, 21.6, 114.0], # 31, 32, 33
        psy = [30.5, 23.4, 125.0], # 34, 35, 36
        cam4_psx = 121.8, # 37
        cam4_psy = 120.0, # 38
        light_amp  = [1.0 .. 13.0 , 1.0 .. 17.0, 1.0 .. 5.0], # 1.0 .. 5.0
        s_cam = [0.0, 1.478, 15.026, 23.1150],
    )


labels = [L"\sigma_x", L"\sigma_y", L"\sigma_x", L"\sigma_y", L"\sigma'_x", L"\sigma'_y", L"\sigma'_x", L"\sigma'_y", L"\alpha", L"s", L"s", 
    L"\mu_x", L"\mu_x", L"\mu_x", L"\mu_x", L"\mu_y", L"\mu_y", L"\mu_y", L"\mu_y", 
    "ped", "lf", "a", "res x", "res y", "la", "la", "la"];

In [4]:
smpl_tmp = bat_read(FPATH*"hc-2.hdf5")
smpl_tmp = varshape(prior).(smpl_tmp);

In [5]:
filenames = readdir("../../data/sampling_results/Benchmark-5/")
images = load("../../data/experiment/dataset_2/m2/images-satur.jld2")
hc_mask = [i[1:3] == "hc-" for i in filenames]
hc_fnames = filenames[hc_mask]
hc_eind = parse.(Int64, [i[4:end-5] for i in hc_fnames])
hc_charge = images["charge"][hc_eind]
hc_runind = images["ev_ind"][hc_eind];

In [6]:
filenames = readdir("../../data/sampling_results/Benchmark-5/")
images = load("../../data/experiment/dataset_2/m1/images-satur.jld2")
lc_mask = [i[1:3] == "lc-" for i in filenames]
lc_fnames = filenames[lc_mask]
lc_eind = parse.(Int64, [i[4:end-5] for i in lc_fnames])
lc_charge = images["charge"][lc_eind]
lc_runind = images["ev_ind"][lc_eind];

In [7]:
fnames = [lc_fnames; hc_fnames]
charges = Float64[lc_charge; hc_charge]
runind = Int64[lc_runind ; hc_runind]
charges_n = charges ./ maximum(charges)
runind_n = runind ./ maximum(runind);

In [8]:
msklc = 1:length(lc_fnames)
mskhc = length(lc_fnames)+1:length(lc_fnames)+length(hc_fnames);

In [9]:
charges_n[sortperm(charges_n)] .= 1:length(charges_n)
charges_n = charges_n ./ maximum(charges_n);

In [10]:
mean_params =  [marg_mean_shaped(i) for i in fnames];
std_params =  [marg_std_shaped(i) for i in fnames];

In [11]:
zc_x = [minimum([i.algmx[j] for i in mean_params]) for j in 1:4]; # find min position at each camera 
zc_y = [minimum([i.algmy[j] for i in mean_params]) for j in 1:4]; # find min position at each camera 

In [18]:
x_event = Array{Float64,1}[]
y_event = Array{Float64,1}[]
x_event_std = Array{Float64,1}[]
y_event_std = Array{Float64,1}[]

for i in eachindex(mean_params) 
    push!(x_event, (mean_params[i].algmx .- zc_x) .* [mean_params[i].psx ; mean_params[i].cam4_psx] .+ 1.0 )
    push!(y_event, (mean_params[i].algmy .- zc_y) .* [mean_params[i].psy ; mean_params[i].cam4_psy] .+ 1.0 )
    
    push!(x_event_std, std_params[i].algmx .* [mean_params[i].psx ; mean_params[i].cam4_psx])
    push!(y_event_std, std_params[i].algmy .* [mean_params[i].psy ; mean_params[i].cam4_psy])
     
end

In [19]:
summary_data = (
    
    ev_index = runind,
    ev_charge = charges,
    
    μx_trsz_halo = Float64[i.tr_size[1] for i in mean_params],
    μy_trsz_halo = Float64[i.tr_size[2] for i in mean_params],
    
    μx_trsz_core = Float64[i.tr_size_2[1] for i in mean_params],
    μy_trsz_core = Float64[i.tr_size_2[2] for i in mean_params],
    
    μx_angspr_halo = Float64[i.ang_spr[1] for i in mean_params],
    μy_angspr_halo = Float64[i.ang_spr[2] for i in mean_params],
    
    μx_angspr_core = Float64[i.ang_spr_2[1] for i in mean_params],
    μy_angspr_core = Float64[i.ang_spr_2[2] for i in mean_params],
    
    μ_waist_halo = Float64[i.waist[1] for i in mean_params],
    μ_waist_core = Float64[i.waist_2[1] for i in mean_params],
    
    μx_align = x_event,
    μy_align = y_event,
    
    σx_trsz_halo = Float64[i.tr_size[1] for i in std_params],
    σy_trsz_halo = Float64[i.tr_size[2] for i in std_params],
    
    σx_trsz_core = Float64[i.tr_size_2[1] for i in std_params],
    σy_trsz_core = Float64[i.tr_size_2[2] for i in std_params],
    
    σx_angspr_halo = Float64[i.ang_spr[1] for i in std_params],
    σy_angspr_halo = Float64[i.ang_spr[2] for i in std_params],
    
    σx_angspr_core = Float64[i.ang_spr_2[1] for i in std_params],
    σy_angspr_core = Float64[i.ang_spr_2[2] for i in std_params],
        
    σ_waist_halo = Float64[i.waist[1] for i in std_params],
    σ_waist_core = Float64[i.waist_2[1] for i in std_params],
    
    σx_align = x_event_std,
    σy_align = y_event_std,
);



In [23]:
# @save "../../data/sampling_results/Benchmark-5/alignment/mcmc-summary-data.jld" summary_data